In [ ]:
# Install Dependencies and Bring in Data
#  Preprocess
# Create a Squential Model
# Evaluate Model
# Test and Gradio

In [ ]:
!pip install tensorflow-gpu==2.9.0


In [ ]:
!pip install tensorflow==2.9.0

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
dataset_path = '/content/drive/My Drive/Datasets/Toxic Comment Classification Challenge/'

In [ ]:
df = pd.read_csv(dataset_path + 'train.csv')

In [ ]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
df.iloc[0]["comment_text"]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [ ]:
df[df.columns[2:]].iloc[3]

toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
Name: 3, dtype: int64

In [ ]:
df.shape

(159571, 8)

In [ ]:
import seaborn as sns

In [ ]:
df[[i for i in df.columns if i  not in ["id", "comment_text"]   ]].value_counts()

In [ ]:
df.isna().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
TextVectorization

keras.src.layers.preprocessing.text_vectorization.TextVectorization

In [ ]:
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [ ]:
X = df["comment_text"]
y = df[df.columns[2:]].values # values to convert this dataframe to arrays

In [ ]:
X

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [ ]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
# how many word we are going to have in  inside our dictionary
MAX_FEATURES = 200000 # number of words in the vocab

In [ ]:
vectorizer = TextVectorization( max_tokens = MAX_FEATURES ,
                               output_sequence_length = 1800 ,
                                output_mode ="int" )

In [ ]:
type(X.values)

numpy.ndarray

In [ ]:
X.values.shape

(159571,)

In [ ]:
vectorizer.adapt(X.values)

In [ ]:
vectorizer.get_vocabulary()

In [ ]:
len(vectorizer.get_vocabulary())

200000

In [ ]:
vectorizer("Hello, world, life is great")

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([288, 263, 306, ...,   0,   0,   0])>

In [ ]:
len(vectorizer("Hello, world, life is great"))

1800

In [ ]:
vectorized_text = vectorizer(X.values)

In [ ]:
vectorized_text.shape

TensorShape([159571, 1800])

In [ ]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [ ]:
# MCSHBAP map , chache, shuffle , batch , prefetch from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices(( vectorized_text , y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) #helps prevent bottlenecks

In [ ]:
dataset.as_numpy_iterator().next()

In [ ]:
batch_X , batch_y = dataset.as_numpy_iterator().next()

In [ ]:
batch_X.shape

(16, 1800)

In [ ]:
batch_y.shape

(16, 6)

In [ ]:
len(dataset) # its kind of number of batches of size 16

9974

In [ ]:
9974*16

159584

In [ ]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip( int(len(dataset)*.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.2))

In [ ]:
train.as_numpy_iterator().next()

In [ ]:
# creating the sequential network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM , Dropout, Bidirectional, Dense, Embedding

In [ ]:
model = Sequential()
# create the embedding layer
model.add( Embedding(MAX_FEATURES +1, 32))
#
model.add( Bidirectional ( LSTM(32, activation = "tanh")))

model.add( Dense(128, activation='relu'))
model.add( Dense(256, activation='relu'))
model.add( Dense(128, activation='relu'))
model.add( Dense(6, activation= 'sigmoid' ))

In [ ]:
model.compile( loss = "BinaryCrossentropy" , optimizer = "Adam")

In [ ]:

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          6400032   
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                16640     
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 256)               33024     
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dense_7 (Dense)             (None, 6)                 774       
                                                      

In [ ]:
history = model.fit(train, epochs =1 , validation_data = val)

Epoch 1/50
6981/6981 [==============================] - 12936s 2s/step - loss: 0.0619 - val_loss: 0.0460
Epoch 2/50
2667/6981 [==========>...................] - ETA: 2:06:51 - loss: 0.0456

In [ ]:
# Save the model to Google Drive
model.save('/content/drive/My Drive/my_model')

In [ ]:
from tensorflow.keras.models import load_model
loaded_model = load_model('/content/drive/My Drive/my_model')

In [ ]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [ ]:
batch = test.as_numpy_iterator().next()

In [ ]:
test.as_numpy_iterator().next()

In [ ]:
batch_X , batch_y = test.as_numpy_iterator().next()

In [ ]:
(loaded_model.predict(batch_X)>0.5).astype(int)

In [ ]:
batch_y

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():

  X_true , y_true = batch
  yhat = loaded_model.predict(X_true)

  y_true = y_true.flatten()
  yhat = yhat.flatten()

  pre.update_state( y_true, yhat)
  re.update_state( y_true, yhat)
  acc.update_state( y_true, yhat)

In [ ]:
print("Precision :" , pre.result().numpy())
print("Recall :" , re.result().numpy())
print("Accuracy :" , acc.result().numpy())

Precision : 0.8717949
Recall : 0.8031496
Accuracy : 0.5210421
